In [1]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import DataLoader, TensorDataset
import json
import pdb
import logging.handlers
import argparse
import os
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from networkx import DiGraph, relabel_nodes, all_pairs_shortest_path_length
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, h_recall_score, h_precision_score, fill_ancestors, multi_labeled
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm
from ordered_set import OrderedSet

c:\Users\ASUS\envs\virtual\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ASUS\AppData\Local\Temp\ipykernel_18520\1759466101.py:17: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
file_path = 'train.json'
# Read the JSON file into a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data_train = json.load(file)

rows = []
possible_labels = OrderedSet(label for entry in data_train for label in entry['labels'])

for entry in data_train:
    text = entry['text']
    labels = entry['labels']
    row = {}
    row['text'] = text

    for label in possible_labels:
        row[label] = int(label in labels)

    rows.append(row)

df = pd.DataFrame(rows)
data_train = df

# Check and replace NaN values
for i in data_train.index:
    if pd.isna(data_train["text"][i]):
        data_train["text"][i] = ""




In [3]:
file_path = 'train.json'
# Read the JSON file into a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data_train = json.load(file)

rows = []
possible_labels = OrderedSet(label for entry in data_train for label in entry['labels'])

for entry in data_train:
    text = entry['text']
    labels = entry['labels']
    row = {}
    row['text'] = text

    for label in possible_labels:
        row[label] = int(label in labels)

    rows.append(row)

df = pd.DataFrame(rows)
data_train = df

# Check and replace NaN values
for i in data_train.index:
    if pd.isna(data_train["text"][i]):
        data_train["text"][i] = ""

In [4]:
file_path = 'validation.json'
# Read the JSON file into a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data_validation = json.load(file)

rows = []
possible_labels = OrderedSet(label for entry in data_validation for label in entry['labels'])

for entry in data_validation:
    text = entry['text']
    labels = entry['labels']
    row = {}
    row['text'] = text

    for label in possible_labels:
        row[label] = int(label in labels)

    rows.append(row)

df = pd.DataFrame(rows)
data_validation = df

# Check and replace NaN values
for i in data_validation.index:
    if pd.isna(data_validation["text"][i]):
        data_validation["text"][i] = ""

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(data_train["text"]), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(list(data_validation['text']), truncation=True, padding=True, return_tensors='pt')

In [6]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels.values, dtype=torch.float32)

    def __getitem__(self, idx):
        return {key: val[idx].to(device) for key, val in self.encodings.items()}, self.labels[idx].to(device)

    def __len__(self):
        return len(self.labels)



In [7]:
labels = sorted([col for col in data_train.columns if col not in ["text", "id"]])

train_dataset = CustomDataset(train_encodings, data_train[labels])
test_dataset = CustomDataset(test_encodings, data_validation[labels])
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(labels)).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from networkx import DiGraph
from torch import nn

G = DiGraph()
G.add_edge(ROOT, "Logos")
G.add_edge("Logos", "Repetition")
G.add_edge("Logos", "Obfuscation, Intentional vagueness, Confusion")
G.add_edge("Logos", "Reasoning")
G.add_edge("Logos", "Justification")
G.add_edge('Justification', "Slogans")
G.add_edge('Justification', "Bandwagon")
G.add_edge('Justification', "Appeal to authority")
G.add_edge('Justification', "Flag-waving")
G.add_edge('Justification', "Appeal to fear/prejudice")
G.add_edge('Reasoning', "Simplification")
G.add_edge('Simplification', "Causal Oversimplification")
G.add_edge('Simplification', "Black-and-white Fallacy/Dictatorship")
G.add_edge('Simplification', "Thought-terminating cliché")
G.add_edge('Reasoning', "Distraction")
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', "Presenting Irrelevant Data (Red Herring)")
G.add_edge('Distraction', "Whataboutism")
G.add_edge(ROOT, "Ethos")
G.add_edge('Ethos', "Appeal to authority")
G.add_edge('Ethos', "Glittering generalities (Virtue)")
G.add_edge('Ethos', "Bandwagon")
G.add_edge('Ethos', "Ad Hominem")
G.add_edge('Ethos', "Transfer")
G.add_edge('Ad Hominem', "Doubt")
G.add_edge('Ad Hominem', "Name calling/Labeling")
G.add_edge('Ad Hominem', "Smears")
G.add_edge('Ad Hominem', "Reductio ad hitlerum")
G.add_edge('Ad Hominem', "Whataboutism")
G.add_edge(ROOT, "Pathos")
G.add_edge('Pathos', "Exaggeration/Minimisation")
G.add_edge('Pathos', "Loaded Language")
G.add_edge('Pathos', "Appeal to (Strong) Emotions")
G.add_edge('Pathos', "Appeal to fear/prejudice")
G.add_edge('Pathos', "Flag-waving")
G.add_edge('Pathos', "Transfer") 

In [9]:
class HierarchicalLoss(nn.Module):
    def __init__(self, alpha=1.0):
        super(HierarchicalLoss, self).__init__()
        self.alpha = alpha

    def forward(self, logits, labels, hierarchy):
        loss = nn.BCEWithLogitsLoss()(logits, labels)

        # Add hierarchical regularization
        for i in range(labels.size(1)):
            for j in range(labels.size(1)):
                if i != j and hierarchy.has_node(i) and hierarchy.has_node(j) and nx.has_path(hierarchy, source=i, target=j):
                    loss += self.alpha * torch.relu(logits[:, i] - logits[:, j])

        return loss

In [10]:
hierarchical_loss = HierarchicalLoss(alpha=0.1)

num_epochs = 15


model.to(device)

for epoch in tqdm(range(num_epochs)):
    model.train()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs, labels = batch
        outputs = model(**inputs).logits
        loss = hierarchical_loss(outputs, labels, G)
        loss.backward()
        optimizer.step()
    torch.save(model, "roberta_subtask1_epoch_" + str(epoch) + ".pt")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = torch.sigmoid(model(**inputs).logits)
        all_preds.extend(outputs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

threshold = 0.5
binary_preds = (np.array(all_preds) > threshold).astype(int)

accuracy = accuracy_score(all_labels, binary_preds)
print(f"Accuracy: {accuracy:.2f}")

classification_report_str = classification_report(all_labels, binary_preds, target_names=data_train.columns[1:])
print("Classification Report:\n", classification_report_str)

  0%|          | 0/15 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print("GPU devices:", physical_devices)


GPU devices: []


In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Current CUDA device:", torch.cuda.current_device())
print("CUDA device count:", torch.cuda.device_count())




CUDA available: False


AssertionError: Torch not compiled with CUDA enabled